In [4]:
import os
import zipfile
import tensorflow as tf
from os import path, getcwd, chdir
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "happy-or-sad.zip"

path = "happy-or-sad.zip"
zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

--2021-05-10 06:07:12--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.223.240, 216.58.223.208, 2c0f:fb50:4003:802::2010, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.223.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘happy-or-sad.zip’

happy-or-sad.zip    100%[===================>]   2.55M  1.70MB/s    in 1.5s    

2021-05-10 06:07:14 (1.70 MB/s) - ‘happy-or-sad.zip’ saved [2670333/2670333]



In [7]:
def train_happy_sad_model():

    DESIRED_ACCURACY = 0.999

    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if logs.get('acc') and (logs.get('acc')>DESIRED_ACCURACY):
                print("\nReached 99% accuracy so cancelling training!")
                self.model.stop_training = True

    callbacks = myCallback()
    
    model = tf.keras.models.Sequential([
        Conv2D(16, 3, activation="relu", input_shape=(150, 150, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(32, 3, activation="relu"),
        MaxPooling2D((2, 2)),
        Conv2D(64, 3, activation="relu"),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(1, activation="sigmoid")
    ])

    model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    
    


    train_datagen = ImageDataGenerator(rescale = 1./255)
    train_dir = "/tmp/h-or-s"
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size = (150,150),
        batch_size =  128,
        class_mode = "binary"
        )
    
    history = model.fit_generator(
        train_generator,
        steps_per_epoch = 8,
        epochs = 10,
        validation_steps = 8,
        verbose = 2,
        callbacks = [callbacks])
    # model fitting
    return history.history['accuracy'][-1]

In [8]:
train_happy_sad_model()

Found 80 images belonging to 2 classes.
Epoch 1/10
8/8 - 1s - loss: 0.6899 - accuracy: 0.5000


0.5